In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
import os
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import shutil

c:\Users\Dell\anaconda3\envs\ml\Lib\site-packages\torch\utils\_pytree.py:173: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [14]:
print(os.path.exists(r'\\train'))

False


### Random choose 350 images from each class to train -> avoid Bias


In [29]:
list_dir = os.listdir('CNN_BASICS\\original_images')
for dir in list_dir:
    if dir== 'trash': continue
    files=os.listdir('CNN_BASICS\\original_images\\'+ dir)
    train_path, test_path=train_test_split(files, train_size=350/len(files))
    for path in train_path:
        shutil.copy(os.path.join('CNN_BASICS\\original_images', dir, path), os.path.join('CNN_BASICS\\train', dir))
    for path in test_path:
        shutil.copy(os.path.join('CNN_BASICS\\original_images', dir, path), os.path.join('CNN_BASICS\\test', dir))
    

In [ ]:
# Define transforms for images
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((32, 32)),
    transforms.ToTensor()   #has done normalizing in range [0, ...1]
])

In [15]:
train_set= dsets.ImageFolder(root=r'.\train', transform=transform)
test_set= dsets.ImageFolder(root=r'.\test', transform=transform)

In [16]:
len(train_set)

1750

In [17]:
len(test_set)

641

In [40]:
batch_size = 100
n_iters = 5000  # number of times running 1 batch (in summation train:s 3000 batches)
num_epochs = n_iters / (len(train_set) / batch_size) # run whole data 100 times
num_epochs = int(num_epochs)

# Divide data into batch to iterate over
train_loader = torch.utils.data.DataLoader(dataset=train_set, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_set, 
                                          batch_size=batch_size, 
                                          shuffle=True)

In [41]:
# Info of each batch
# samples x channels (1 or 3 RGB) x height x width
for inputs, labels in train_loader:
    print("Inputs shape:", inputs.shape)
    print("Labels shape:", labels.shape)
    break  # Inspect only the first batch


Inputs shape: torch.Size([100, 1, 32, 32])
Labels shape: torch.Size([100])


### Step 3: Create Model Class

#### Output Formula for Convolution
- $ O = \frac {W - K + 2P}{S} + 1$
    - $O$: output height/length
    - $W$: input height/length
    - $K$: **filter size (kernel size) = 5**
    - $P$: **same padding (non-zero)**
        - $P = \frac{K - 1}{2}  = \frac{5 - 1}{2} = 2$
    - $S$: **stride = 1**
    
#### Output Formula for Pooling/Downsampling
- $ O = \frac {W - K}{S} + 1$
    - W: input height/width
    - K: **filter size = 2**
    - S: **stride size = filter size**, PyTorch defaults the stride to kernel filter size
        - If using PyTorch default stride, this will result in the formula $ O = \frac {W}{K}$
    
<img src="./images/cnn10-2n.png" alt="deeplearningwizard" style="width: 900px;"/>

In [46]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
                            #color channels # 16 kernel types
                                            # kernels are initialized randomly and are adjusted through backpropagation.
                                            # choosing number of kernels depends on experience
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # # Convolution 2
        # self.cnn2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        # self.relu2 = nn.ReLU()
        
        # # Max pool 2
        # self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1 (readout - dense layer)
        self.fc1 = nn.Linear(16 * 16 * 16, 5) 
    
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # # Convolution 2 
        # out = self.cnn2(out)
        # out = self.relu2(out)
        
        # # Max pool 2 
        # out = self.maxpool2(out)
        
        # Resize
        # Original size: (100, 32, 8, 8)
        # New out size: (100, 32*8*8)
        out = out.view(out.size(0), -1)

        # Linear function (readout)
        # Put 50 output samples into linear to calc softmax and cross entropy loss for each output sample
        out = self.fc1(out)
        
        return out

In [47]:
model1= CNNModel()
criterion = nn.CrossEntropyLoss()

learning_rate = 0.01
optimizer = torch.optim.SGD(model1.parameters(), lr=learning_rate)  

In [43]:
# Đưa model vào GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [48]:
total_params = sum(p.numel() for p in model1.parameters())
total_params


20901

In [49]:
'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        
        # Feed input to GPU
        images = images.requires_grad_().to(device)
        labels = labels.to(device)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model1(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                images = images.requires_grad_().to(device)
                labels = labels.to(device)
                
                # Forward pass only to get logits/output
                outputs = model1(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same